<a href="https://colab.research.google.com/github/nagusubra/AI-customer-counter/blob/main/Solar_panel_dust_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install libraries and modules

In [1]:
!pip install -q tensorflow-model-optimization

import tempfile
from tensorflow import keras
import tensorflow_model_optimization as tfmot

from tqdm import tqdm_notebook as tqdm
import os
import time
import pickle
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 6.2 MB/s eta 0:00:00


In [2]:
# mounting google drive (if you are using Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# from os import listdir
# from os.path import isfile, join

# mypath = '/content/drive/MyDrive/Solar_panel_dust_detection/dataset_2/dirty'

# onlyfiles = [ f  for f in listdir(mypath) if isfile(join(mypath, f))]

# onlyfiles

In [4]:
# from os import listdir
# from os.path import isfile, join

# mypath = '/content/drive/MyDrive/Solar_panel_dust_detection/dataset_2/dirty'

# onlyfiles = [ os.rename(join(mypath, f), join(mypath, f.split(".")[0] + "_dirty" + ".jpg"))  for f in listdir(mypath) if isfile(join(mypath, f))]

# onlyfiles

# Build base model

In [10]:
def SolNet(in_size):
  i = Input(in_size)
  r = Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3))(i)
  r = BatchNormalization()(r)
  r = MaxPooling2D(pool_size=(3,3), strides=(2,2))(r)
  r = Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same")(r)
  r = BatchNormalization()(r)
  r = MaxPooling2D(pool_size=(3,3), strides=(2,2))(r)
  r = Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(r)
  r = BatchNormalization()(r)
  r = Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(r)
  r = BatchNormalization()(r)
  r = Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same")(r)
  r = BatchNormalization()(r)
  r = MaxPooling2D(pool_size=(3,3), strides=(2,2))(r)
  r = Flatten()(r)
  r = Dense(4096, activation='relu')(r)
  r = Dropout(0.5)(r)
  r = Dense(4096, activation='relu')(r)
  r = Dropout(0.5)(r)
  o = Dense(1, activation='sigmoid')(r)
  SolNet = Model(i, o)
  SolNet.save('models/solnet.hdf5')
  SolNet.compile(optimizer=Adam(.0001, .8, .9), loss=binary_crossentropy, metrics=['acc'])
  SolNet.summary()
  return SolNet

# Load dataset

In [5]:

directory_path = "/content/drive/MyDrive/Solar_panel_dust_detection/dataset_1"

image_dateset =  tf.keras.utils.image_dataset_from_directory(
                                                              directory_path,
                                                              labels="inferred",
                                                              label_mode="categorical",
                                                              class_names=None,
                                                              color_mode="rgb",
                                                              batch_size=32,
                                                              image_size=(256, 256),
                                                              shuffle=True,
                                                              seed=None,
                                                              validation_split=None,
                                                              subset=None,
                                                              interpolation="bilinear",
                                                              follow_links=False,
                                                              crop_to_aspect_ratio=False
                                                          )

Found 788 files belonging to 2 classes.


In [8]:

directory_path = "/content/drive/MyDrive/Solar_panel_dust_detection/dataset_1"

image_dateset_1 =  tf.keras.utils.image_dataset_from_directory(
                                                              directory_path,
                                                              labels="inferred",
                                                              label_mode=None,
                                                              class_names=None,
                                                              color_mode="rgb",
                                                              batch_size=32,
                                                              image_size=(256, 256),
                                                              shuffle=True,
                                                              seed=None,
                                                              validation_split=None,
                                                              subset=None,
                                                              interpolation="bilinear",
                                                              follow_links=False,
                                                              crop_to_aspect_ratio=False
                                                          )

Found 788 files belonging to 1 classes.


In [9]:
display(image_dateset)
display(image_dateset_1)

<BatchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

<BatchDataset element_spec=TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None)>